In [1136]:
# IMPORT THE LIBRARIES AND OPEN THE CSV FILE AS df

In [1137]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
import re
from numpy import NaN
from pandas import isnull


df_raw = pd.read_csv ('./data/attacks.csv', encoding='unicode_escape')

# I want to check out the columns that appear in the original dataset (.csv)
df_raw.columns


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [1138]:
'''I remove the columns that I don't want -> ['Investigator or Source', 'pdf', 'href formula', 'href',
'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22','Unnamed: 23'] 
-> I additionally remove all lines that have null information or more than 2 fields that are null
'''

df= df_raw.drop(columns=['Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'])
df = df.dropna(how="all")
df = df.dropna(thresh=3)

df.rename(columns = {"Species ":"Species"}, inplace=True)
df.columns


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species'],
      dtype='object')

In [1139]:
# I want to check the data types of each item
df.dtypes

Case Number     object
Date            object
Year           float64
Type            object
Country         object
Area            object
Location        object
Activity        object
Name            object
Sex             object
Age             object
Injury          object
Fatal (Y/N)     object
Time            object
Species         object
dtype: object

In [1140]:
# I start cleaning the Year Column
# Filter - I will keep year > 1900 and will remove data with no Year associated
# I change year dtype to INT

df.drop (df[df.Year < 1900].index, inplace=True)
df = df[df.Year.notna()]
df["Year"] = df["Year"].astype(int)
df.dtypes

Case Number    object
Date           object
Year            int64
Type           object
Country        object
Area           object
Location       object
Activity       object
Name           object
Sex            object
Age            object
Injury         object
Fatal (Y/N)    object
Time           object
Species        object
dtype: object

In [1141]:
# I will clean Countries - I will put everything in CAPITAL LETTERS so to remove duplicates with countries

df["Country"] = df["Country"].str.upper()
df["Country"].unique()


array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, BRITISH OVERSEAS TERRITORY', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'FIJI', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'PUERTO RICO', 'ITALY',
       'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA',
       'RUSSIA', 'TURKS & CAICOS', 'UNITED ARAB E

In [1142]:
# We will clean the Type column - there are 2 blank values and one value that is numeric (val = "34")

df = df[df.Type.notna()]
df.drop (df[df.Type == "34"].index, inplace=True)



In [1143]:
# Date --> next column

#re.search  -> returns True or False

#if starts with Reported -> return the rest of the date
#I will create a new column --> Month (using regex)
#I will create a new column Day --> based on Date column
#I will create a new column Season --> based on new column Month

#(outcome)  --> ["Day"],["Month"],["Year"],["Season"]

from random import randint


df["Date"] = df["Date"].str.replace("Reported ","")

df.insert(2,"Months",df['Case Number'].str.extract('\.(\d{2})\.'))
df.insert(2,"Day",df['Case Number'].str.extract('\.\d{2}\.(\d{2})'))

#as I have the data separated by ["Days"], ["Months"], ["Year"] I will remove the "Date" column

df.drop(["Date"], axis=1 ,inplace=True)

# I want to turn NaN to "00" so that all missing data on date is grouped together -> i will change data type of columns to INT

df.loc[df["Months"].isnull()]="00"
df.loc[df["Day"].isnull()] ="00"


df["Year"] = df["Year"].astype(int)
df["Months"] = df["Months"].astype(int)
df["Day"] = df["Day"].astype(int)



In [1144]:
print(df.dtypes)
print(df["Months"].unique())
print(df["Day"].unique())

Case Number    object
Day             int64
Months          int64
Year            int64
Type           object
Country        object
Area           object
Location       object
Activity       object
Name           object
Sex            object
Age            object
Injury         object
Fatal (Y/N)    object
Time           object
Species        object
dtype: object
[ 6  5  4  3  2  1 12 11 10  9  8  7  0]
[25 18  9  8  4  3 27 26 24 21 13  0 12 30 28 23 22 19 15 14 10  5 31 17
 11  1 20 16  6  2 29  7]


In [1145]:
south = pd.read_csv ('./data/south.csv', encoding='unicode_escape')
south.dtypes
# column name --> ï»¿SOUTH COUNTRY 

list_south = []
for elem in south["ï»¿SOUTH COUNTRY"]:
    list_south.append(elem)


#outcome --> list of southern hemisphere countries

#I create column Hemisphere


df["Hemisphere"] = np.where(df['Country'].isin(list_south), 'SOUTH', 'NORTH')



In [1146]:
# I will create the season column (based on the hemisphere and the month)

season_list = []
list_hemis = list(df["Hemisphere"])
list_month = list(df["Months"])

for h,m in zip(list_hemis,list_month):
    if h == "NORTH":
        if m in range(1,4):
            season_list.append("WINTER")
        elif m in range(4,7):
            season_list.append("SPRING")
        elif m in range(7,10):
            season_list.append("SUMMER")
        elif m in range(10,13):
            season_list.append("AUTUMN")
        else: 
            season_list.append("NOT DEFINED")

    elif h == "SOUTH":
        if m in range(1,4):
            season_list.append("SUMMER")
        elif m in range(4,7):
            season_list.append("AUTUMN")
        elif m in range(7,10):
            season_list.append("WINTER")
        elif m in range(10,13):
            season_list.append("SPRING")
        else: 
            season_list.append("NOT DEFINED")

df["Season"] = season_list

#i will change the order of teh columns!

df= df.reindex(columns=['Case Number', 'Day', 'Months', 'Year','Season', 'Type', 'Hemisphere','Country', 'Area','Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)','Time', 'Species'])
df.head()


,Case Number,Day,Months,Year,Season,Type,Hemisphere,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,2018.06.25,25,6,2018,SPRING,Boating,NORTH,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,2018.06.18,18,6,2018,SPRING,Unprovoked,NORTH,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,2018.06.09,9,6,2018,SPRING,Invalid,NORTH,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,2018.06.08,8,6,2018,AUTUMN,Unprovoked,SOUTH,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,2018.06.04,4,6,2018,SPRING,Provoked,NORTH,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


In [1149]:
df.rename(columns = {"Sex ":"Sex"}, inplace=True)

df["Sex"] = np.where(df["Sex"] == "00", np.nan, df["Sex"])
df["Sex"] = np.where(df["Sex"] == "M ", "M", df["Sex"])
df["Sex"] = np.where(df["Sex"] == "lli", np.nan, df["Sex"])
df["Sex"] = np.where(df["Sex"] == ".", np.nan, df["Sex"])
df["Sex"] = np.where(df["Sex"] == "N", np.nan, df["Sex"])
df["Sex"].value_counts()


M    4452
F     600
Name: Sex, dtype: int64

In [1150]:
df

,Case Number,Day,Months,Year,Season,Type,Hemisphere,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,2018.06.25,25,6,2018,SPRING,Boating,NORTH,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,2018.06.18,18,6,2018,SPRING,Unprovoked,NORTH,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,2018.06.09,9,6,2018,SPRING,Invalid,NORTH,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,2018.06.08,8,6,2018,AUTUMN,Unprovoked,SOUTH,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,2018.06.04,4,6,2018,SPRING,Provoked,NORTH,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,1900.07.14,14,7,1900,SUMMER,Invalid,NORTH,USA,Hawaii,"Makapu'u Point, O'ahu",Hunting seashells,Emil Uhlbrecht & unidentified person,M,NaN,"Believed drowned. Uhlbrechts foot, and the pe...",NaN,NaN,Questionable
5560,1900.07.00,0,7,1900,SUMMER,Provoked,NORTH,USA,Connecticut,"Bridgeport, Fairfield County",,"skiff with Dr. William T. Healey, Dr. Henry Ca...",NaN,NaN,"No injury to occupants. They shot shark, then ...",N,NaN,NaN
5561,1900.01.28,28,1,1900,SUMMER,Unprovoked,SOUTH,AUSTRALIA,New South Wales,"Lane Cove River, Sydney Harbor (Estuary)","Standing, gathering oysters",Charles Duck,M,NaN,Right posterior thigh bitten,N,12h00,NaN
5562,1900.00.00.b,0,0,1900,NOT DEFINED,Unprovoked,NORTH,USA,Hawaii,"Inter-Island Dry Dock at Kakaako Street, Honol...",NaN,Emil A. Berndt,M,NaN,Severe abrasion when shark swam between his legs,N,NaN,NaN
